In [24]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

El circuito de Chua es un modelo no lineal y a determinados valores genera caos y produce un sinnúmero de comportamientos, incluso toda una familia de atractores extraños y consiste en un oscilador (un condensador y bobina en paralelo), una resistencia en serie con el oscilador y un condensador en paralelo con un dispositivo  no lineal alimentado por alguna fuente externa de potencia comúnmente llamado diodo de Chua.Este diodo esta caracterizado por una respuesta de corriente contra voltaje no lineal, cuya pendiente debe ser negativa en algún lugar de la curva que,un elemento de este tipo es también llamado resistor activo localmente, aunque dicha curva de corriente contra voltaje puede tener muchas formas, el circuito de Chua original, especifica una curva impar, lineal por partes.

El circuito de Chua en su conjunto nos da una curva de transferencia la cuál la podemos obtener por medio de otros circuitos electrónicos. Es por eso que el circuito de Chua tiene una gran importancia ya que es el ejemplo de un sistema caótico que emula  un sistema real físico y comprobado en el laboratorio.

Un aspecto importante es que el circuito de Chua es un sistema que presenta diferentes dinámicas, caóticas oscilatorias y puede ser representado por un sistema de ecuaciones diferenciales las cuales son:

$$\frac{dx}{dt}=\alpha(y-x-f(x))$$

$$\frac{dy}{dt}= x-y+z$$

$$\frac{dz}{dt}= -\beta y - \gamma z$$

$$f(x) = bx + 0.5 (a-b)((\sqrt{x+1})^2-(\sqrt{x-1})^2)$$

Podemos simplificar las ecuaciones usando la curva de transferencia del sistema y aproximarla a una ecuación polinomial de tercer grado. De esta forma podemos llegar a las siguiente ecuaciones:

$$\frac{dx}{dt}=\alpha(y-f(x))$$

$$\frac{dy}{dt}= x-y+z$$

$$\frac{dz}{dt}= -\beta y $$

$$f(x) = \frac{1}{16}x^3-\frac{1}{6}x$$

El parametró gamma ya no existe, este sistema tiene la facilidad de permitir que se hallen sus puntos de equilibrio facilmente.

Los puntos de equilibrio del sistema son 3 incluyendo el trivial, estos se muestran en el siguiente arreglo.

$$\Biggl(\begin{smallmatrix}
x&y&z\\ 0&0&0\\-2.633&0&2.633\\2.633&0&-2.633
\end{smallmatrix} \Biggr)$$

Y podemos encontrar su matriz jacobiana la cual está dada por:

$$\Biggl(\begin{smallmatrix}
\alpha(\frac{3}{16}x^2-\frac{1}{6})&\alpha&0\\ 1&-1&1\\0&-\beta&0
\end{smallmatrix} \Biggr)$$

De la matriz jacobiana se pueden hallar los autovalores en cada punto de equilibrio, para valores de alpha=9.5 y betha=14.

Para el punto de equilibrio trivial sus autovalores son:

$$\Biggl(\begin{smallmatrix}
-1.03+2.70i&0&0\\ 0&2.64&0\\0&0&-1.03+2.70i
\end{smallmatrix} \Biggr)$$

En estos se puede observar dos autovalores complejos conjugados con parte real negativa, que denotan que el punto es oscilatoriamente estable, y un tercer autovalor real positivo que denota la existencia de una variedad inestable de gran peso cerca al punto de equilibrio.

Para segundo punto de equilibrio con x negativa y z positiva, a partir de sus valores propios todos con parte real negativa se puede decir que es un punto de equilibrio completamente estable.

$$\Biggl(\begin{smallmatrix}
-11.57&0&0\\ 0&-0.09-3.60i&0\\0&0&-0.09+3.60i
\end{smallmatrix} \Biggr)$$

El tercer punto de equilibrio tiene iguales características que
el segundo al tener igual juego de autovalores.

El sistema con los parametros tomando los valores discutidos en esta ultimá parte se comporta de la siguiente manera:

In [25]:
def chua_deriv((x, y, z), t0, alpha=9.5 , beta=14.): # definine el sistema de chua
    #se utilizo una generalización que sustituye la función continua lineal a tramos f( x )
    #por una función suave, tal como un  polinomio cubico, en este caso fue estudiado por Hirsch y Smale
    f = ((1./16.)*x**3 - (1./6.)*x)
    return [alpha * (y  - f),  x - y + z, -(beta * y)]

In [32]:
N_trajectories = 20

# Elije puntos de inicio al azar, distribuidos uniformemente en torno al punto de equilibrio trivial (0,0,0)
np.random.seed(1)
x0 =  .0001 * np.random.random((N_trajectories, 3))

# Resuelve para las trayectorias
t = np.linspace(0, 100, 10000)
x_t = np.asarray([integrate.odeint(chua_deriv, x0i, t)
                  for x0i in x0])

In [33]:
# Establece la figura y eje 3D para la animación 
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('on')

# Elege un color diferente para cada trayectoria 
colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

# Establece líneas y puntos 
lines = sum([ax.plot([], [], [], '-', c=c)
             for c in colors], [])
pts = sum([ax.plot([], [], [], 'o', c=c)
           for c in colors], [])

# Prepara el límite de los ejes 
zoom = 3
ax.set_xlim((-zoom, zoom))
ax.set_ylim((-zoom, zoom))
ax.set_zlim((-zoom, zoom))

# Conjunto de puntos de vista: especificado por (grados de altitud, grados  azimutales) 
ax.view_init(30, 0)

# Función de inicialización: trama del fondo de cada trama
def init():
    for line, pt in zip(lines, pts):
        line.set_data([], [])
        line.set_3d_properties([])

        pt.set_data([], [])
        pt.set_3d_properties([])
    return lines + pts

# Función de animación. Esto se llama secuencialmente con el número de cuadro 
def animate(i):
    # un paso por 6 pasos de tiempo por trama. Esto conduce a buenos resultados.
    i = (6 * i) % x_t.shape[1]

    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        line.set_data(x, y)
        line.set_3d_properties(z)

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])

    ax.view_init(30, 0.3 * i) # angulo de vista y velocidad de rotacion
    fig.canvas.draw()
    return lines + pts

# inicializa el animador
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=5000, interval=30, blit=True)

# Guardar como MP4. Esto requiere mplayer o ffmpeg instalado. 
# anim.save ( 'lorentz_attractor.mp4', fps = 15, extra_args = ['- vcodec', 'libx264'])

plt.show()

In [34]:
# Elije puntos de inicio al azar, distribuidos uniformemente en torno al punto de equilibrio (-2.633,0,2.633)
np.random.seed(1)
x0 =  (-2.633,0,2.633)+ .0001*np.random.random((N_trajectories,3))

# Resuelve para las trayectorias
t = np.linspace(0, 100, 10000)
x_t = np.asarray([integrate.odeint(chua_deriv, x0i, t)
                  for x0i in x0])

In [35]:
# Establece la figura y eje 3D para la animación 
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('on')

# Elege un color diferente para cada trayectoria 
colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

# Establece líneas y puntos 
lines = sum([ax.plot([], [], [], '-', c=c)
             for c in colors], [])
pts = sum([ax.plot([], [], [], 'o', c=c)
           for c in colors], [])

# Prepara el límite de los ejes 
zoom = 3
ax.set_xlim((-zoom, zoom))
ax.set_ylim((-zoom, zoom))
ax.set_zlim((-zoom, zoom))

# Conjunto de puntos de vista: especificado por (grados de altitud, grados  azimutales) 
ax.view_init(30, 0)

# Función de inicialización: trama del fondo de cada trama
def init():
    for line, pt in zip(lines, pts):
        line.set_data([], [])
        line.set_3d_properties([])

        pt.set_data([], [])
        pt.set_3d_properties([])
    return lines + pts

# Función de animación. Esto se llama secuencialmente con el número de cuadro 
def animate(i):
    # un paso por 6 pasos de tiempo por trama. Esto conduce a buenos resultados.
    i = (6 * i) % x_t.shape[1]

    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        line.set_data(x, y)
        line.set_3d_properties(z)

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])

    ax.view_init(30, 0.3 * i) # angulo de vista y velocidad de rotacion
    fig.canvas.draw()
    return lines + pts

# inicializa el animador
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=5000, interval=30, blit=True)

# Guardar como MP4. Esto requiere mplayer o ffmpeg instalado. 
# anim.save ( 'lorentz_attractor.mp4', fps = 15, extra_args = ['- vcodec', 'libx264'])

plt.show()

In [36]:
# Elije puntos de inicio al azar, distribuidos uniformemente en torno al punto de equilibrio  (2.633,0,-2.633)
np.random.seed(1)
x0 =  (2.633,0,-2.633)+ .001*np.random.random((N_trajectories,3))

# Resuelve para las trayectorias
t = np.linspace(0, 100, 10000)
x_t = np.asarray([integrate.odeint(chua_deriv, x0i, t)
                  for x0i in x0])

In [37]:
# Establece la figura y eje 3D para la animación 
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('on')

# Elege un color diferente para cada trayectoria 
colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

# Establece líneas y puntos 
lines = sum([ax.plot([], [], [], '-', c=c)
             for c in colors], [])
pts = sum([ax.plot([], [], [], 'o', c=c)
           for c in colors], [])

# Prepara el límite de los ejes 
zoom = 3
ax.set_xlim((-zoom, zoom))
ax.set_ylim((-zoom, zoom))
ax.set_zlim((-zoom, zoom))

# Conjunto de puntos de vista: especificado por (grados de altitud, grados  azimutales) 
ax.view_init(30, 0)

# Función de inicialización: trama del fondo de cada trama
def init():
    for line, pt in zip(lines, pts):
        line.set_data([], [])
        line.set_3d_properties([])

        pt.set_data([], [])
        pt.set_3d_properties([])
    return lines + pts

# Función de animación. Esto se llama secuencialmente con el número de cuadro 
def animate(i):
    # un paso por 6 pasos de tiempo por trama. Esto conduce a buenos resultados.
    i = (6 * i) % x_t.shape[1]

    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        line.set_data(x, y)
        line.set_3d_properties(z)

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])

    ax.view_init(30, 0.3 * i) # angulo de vista y velocidad de rotacion
    fig.canvas.draw()
    return lines + pts

# inicializa el animador
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=5000, interval=30, blit=True)

# Guardar como MP4. Esto requiere mplayer o ffmpeg instalado. 
# anim.save ( 'lorentz_attractor.mp4', fps = 15, extra_args = ['- vcodec', 'libx264'])

plt.show()